In [1]:
import pandas as pd
import json
import csv
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import string
import numpy as np
from nltk.corpus import stopwords
import string
import emoji
import string
from datetime import timedelta

In [2]:
dir_="C:\\Users\\Alex\\Personal-Code\\ACE_592_Dating_Sim\\"

In [3]:
#reads the csv files into memory for each dating app.
#Data has already been cleaed and all three files share the same columns
dfT = pd.read_csv(dir_ + "Raw_Data\\tinder_google_play_reviews.csv")
dfB = pd.read_csv(dir_ + "Raw_Data\\bumble_google_play_reviews.csv")
dfH = pd.read_csv(dir_ + "Raw_Data\\hinge_google_play_reviews.csv")

In [4]:
dfH.head(5);

In [5]:
dfB.head(5);

In [6]:
dfT.head(5)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOElIsyH7v-Hhhzu6PuI9DIsN7VUcxx0h5SBKpK...,Nikhilesh Verma,https://play-lh.googleusercontent.com/a-/AOh14...,Fake app,1,0,NaN,2022-04-09 11:15:07,NaN,NaN
1,gp:AOqpTOH0smXMRH_GMXoPk0F6hUbPoaQ6e6a29shjEfS...,Tristan Jones,https://play-lh.googleusercontent.com/a/AATXAJ...,"I'm not a racist person, l respect all races b...",2,0,13.5.0,2022-04-09 11:03:58,NaN,NaN
2,gp:AOqpTOFKzNUWcq_qhxzt2Pv8FsukbMOar4kE8f9m2yD...,Lola Racheal,https://play-lh.googleusercontent.com/a/AATXAJ...,I was banned from using the app i don't know w...,1,0,13.5.0,2022-04-09 11:00:09,NaN,NaN
3,gp:AOqpTOF9FSIUVm7b_o1_SJ1p4zi-WpTxBbToccDf5bN...,Boakiller,https://play-lh.googleusercontent.com/a-/AOh14...,Ive been matching with people but they never s...,1,0,13.5.0,2022-04-09 10:54:40,NaN,NaN
4,gp:AOqpTOE9BAQpxA1lZJpmnzmJIaRPxAkhJrXBHA-cF1b...,Maddison Macfarlane,https://play-lh.googleusercontent.com/a-/AOh14...,"Got banned, don't know why or how? There isn't...",1,0,NaN,2022-04-09 10:46:22,NaN,NaN


In [7]:
dfT.dtypes

reviewId                object
userName                object
userImage               object
content                 object
score                    int64
thumbsUpCount            int64
reviewCreatedVersion    object
at                      object
replyContent            object
repliedAt               object
dtype: object

In [8]:
#cleans and crops the data. Converts the timestamp to a datetime format, and clips the data to only include 2018-2022
def clean(df):
    #throws the copy error warning because it annoys me
    pd.options.mode.chained_assignment = None  # default='warn'
    
    #TIME STUFF
    # Convert to datetime
    df['time_stamp'] = pd.to_datetime(df['at'])
    # Adds Date column, which is just chopping off the time part. Will hopefully be usefull later
    df['date'] = df.time_stamp.apply(lambda x:  str(x)[:10])
    df['date'] = pd.to_datetime(df.date)
    
    #checks on whether or not a review was replied to and stores that bool in a new column "reply_status"
    df['reply_status'] = 0
    for i in range(len(df)):
        if pd.isnull(df.replyContent[i]):
            #print("false")
            df.reply_status[i] = False
        else:
            #print("true")
            df.reply_status[i] = True
    
    #CLEANING THE TEXT (scrub scrub)
    #converts text to string for all review comments 
    df['full_text'] = df['content'].astype(str)
    
    #converts all text to lowercase: 
    df['clean_text'] =  df['full_text'].str.lower().str.replace("\n","")
    
    #removes punctuation
    string.punctuation
    # This maketrans function can be used to remove a list of characters from a string.
    remv_punc = str.maketrans('','',string.punctuation + '“' +"‘"+'”')
    df['clean_text'] = df['clean_text'].str.translate(remv_punc)
    
    #on the off chance that any of these come up in the reviews
    #starts by removing the https, #, and @ AND THEIR FOLLOWING TEXT from our full text tweets 
    #Regex functions to find each one
    find_links = re.compile(r'http\S+')
    find_users = re.compile(r'@\w+')
    find_hashtags = re.compile(r'#\w+')

    #"sub" them for an empty string
    df['clean_text'] = [find_links.sub("",x) for x in df['clean_text']]
    df['clean_text']=  [find_users.sub("",x) for x in df['clean_text']]
    df['clean_text']= [find_hashtags.sub("",x) for x in df['clean_text']]
    
    #renames thumbs and reply_content
    df = df.rename(columns = {'thumbsUpCount' : 'thumbs', 'replyContent' : 'reply_content'})
    
    df1 = df[['userName', 'full_text','clean_text', 'score', 'thumbs', 'date', 'time_stamp', 'reply_status', 'reply_content']]
    
    return df1

In [9]:
dfT = clean(dfT)
dfB = clean(dfB)
dfH = clean(dfH)
print(dfT.dtypes);
dfT.head(5)

userName                 object
full_text                object
clean_text               object
score                     int64
thumbs                    int64
date             datetime64[ns]
time_stamp       datetime64[ns]
reply_status             object
reply_content            object
dtype: object


,userName,full_text,clean_text,score,thumbs,date,time_stamp,reply_status,reply_content
0,Nikhilesh Verma,Fake app,fake app,1,0,2022-04-09,2022-04-09 11:15:07,False,NaN
1,Tristan Jones,"I'm not a racist person, l respect all races b...",im not a racist person l respect all races but...,2,0,2022-04-09,2022-04-09 11:03:58,False,NaN
2,Lola Racheal,I was banned from using the app i don't know w...,i was banned from using the app i dont know wh...,1,0,2022-04-09,2022-04-09 11:00:09,False,NaN
3,Boakiller,Ive been matching with people but they never s...,ive been matching with people but they never s...,1,0,2022-04-09,2022-04-09 10:54:40,False,NaN
4,Maddison Macfarlane,"Got banned, don't know why or how? There isn't...",got banned dont know why or how there isnt eve...,1,0,2022-04-09,2022-04-09 10:46:22,False,NaN


In [11]:
#saves the new CSV files as clean data
dfT.to_csv(dir_ + "Wrangled_Data\\clean_Tinder.csv")
dfB.to_csv(dir_ + "Wrangled_Data\\clean_Bumble.csv")
dfH.to_csv(dir_ + "Wrangled_Data\\clean_Hinge.csv")